In [ ]:
import requests
import time
import re
from bs4 import BeautifulSoup
from bs4 import UnicodeDammit
import http.cookiejar
import os 
import sys
import ijson
import io
from bson.code import Code
import json
import string
from datetime import datetime

### 1.Scrape Nasdaq to get all the news link, title and dynamic api link

In [ ]:
#The website url does not get the data by get request, it calls an api which can be found by inspecting
headers = {
"Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Accept-Encoding":"gzip, deflate",
"Accept-Language":"en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7",
"Connection":"keep-alive",
"Host":"www.nasdaq.com",
"Referer":"http://www.nasdaq.com",
"Upgrade-Insecure-Requests":"1",
'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'} 

result_links=[]
result_titles=[]
for pg in range(1,50): #page number
    api_url = 'https://www.nasdaq.com/api/v1/search?q=tesla&offset='+str((pg-1)*10)  # get dynamic api link
    response=requests.get(api_url,headers=headers)
    json_result = json.loads(response.content) #the result is a json file 
    html_result=json_result['items'] #parse the json file 
    no_of_result=0 #count number of result 
    for el in html_result:
        time.sleep(3) 
        soup=BeautifulSoup(el,'html.parser')  ##parse the html beautifully
        no_of_result=no_of_result+1
        #print(no_of_result," results") ##to help debugging
        
        #find each search result title,link, and timestamp
        result_title_tag=soup.select("h2.search-result__title>a")[0] 
        result_title=result_title_tag.get("title")
        partial_link=result_title_tag.get("href") #link is only partial
        try:
            date_stamp_string=re.search(r'[0-9]{4}-[0-9]{2}-[0-9]{2}',partial_link)[0]
            date_stamp=datetime.strptime(date_stamp_string, '%Y-%m-%d') #transfer to date object
            date_stamp_bchmark=datetime.strptime('2019-10-01','%Y-%m-%d') #2019.10.01 is the time stock price start to soar
        except:
            continue
        
        #skip the symbol results which are not news
        result_eyebrow=soup.select("div.search-result__eyebrow")[0]
        #skip the ones that are not news or the ones that are earlier than 2019.10.01
        if result_eyebrow.text!="Symbols" and date_stamp>=date_stamp_bchmark: 
            full_link="https://www.nasdaq.com"+str(partial_link)
            result_links.append(full_link)
            result_titles.append(result_title)
            print(result_title)
            print(full_link)
    if response.status_code!=200:
        raise ValueError("Invalid Response Received From Webserver")
    #print(result)

## 2. download the links 

In [ ]:
#need to get the cookie 
session_requests = requests.session()
# going to the home page while logged in  
r2=session_requests.get('https://www.nasdaq.com/',headers=headers)
cookie=r2.cookies.get_dict()
print("cookie is :",cookie)

for link in result_links:
    ##download each html  GET requests
    response=requests.get(link,cookies=cookie,headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser') 
#   print(soup)
#   write file
    no_of_news=result_links.index(link)
    title=str(result_titles[no_of_news]).translate(str.maketrans(' ','_',string.punctuation))
    with open("tesla_news_"+str(no_of_news+1)+"_"+str(title)+".htm","w",encoding='utf-8') as file:
        html_unicode=UnicodeDammit(str(soup)).unicode_markup
        file.write(html_unicode)

## 3. Parse and get the content of the news 

In [ ]:
directory = os.getcwd() #get the directory
for filename in os.listdir(directory):
    if filename.endswith('.htm') == False:
        continue
    try:
        with open(os.path.join(directory, filename), 'r',encoding="utf-8") as file:
            text = file.read()
            soup = BeautifulSoup(text, 'html.parser') 
            #print(soup)
            #get paragraph
            paragraphs = soup.findAll('p') 
        if not paragraphs:
            paragraph=None
        else:
            print(filename)
            for el in paragraphs[1:]: #first <p> label is meaningless
                if el.text=="The views and opinions expressed herein are the views and opinions of the author and do not necessarily reflect those of Nasdaq, Inc.":
                    break

                print(el.text) 
    except:
        print("file "+filename,sys.exc_info()[0])